In [1]:
from models.common import DetectMultiBackend

In [2]:
weights = "./yolov5s.pt"

model = DetectMultiBackend(weights)

  0%|          | 0.00/14.1M [00:00<?, ?B/s]

Fusing layers... 


YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients


In [3]:
from utils.datasets import LoadImages

source = "./data/images/bus.jpg"

imgsz = (640,640)
stride = model.stride
pt = model.pt

dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt)

In [5]:
import torch

path, im, im0s, vid_cap, s = next(iter(dataset))

im = torch.from_numpy(im).float()
im /= 255.0
im = im[None]

In [11]:
import torch.nn as nn

class FilterConfidence(nn.Module):
    def __init__(self, conf_thres=0.25):
        super(FilterConfidence, self).__init__()
        self.conf_thres = conf_thres
        
    def forward(self, x):
        idx = x[..., 4] > self.conf_thres
        x = x[idx]
        return x

In [12]:
f = FilterConfidence()

model_with_postprocessing = nn.Sequential(model, f)

In [14]:
torch.onnx.export(model_with_postprocessing, im, "yolo_with_postprocessing.onnx", opset_version=11)

/home/dhuynh95/test/yolov5/models/yolo.py:60: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.onnx_dynamic or self.grid[i].shape[2:4] != x[i].shape[2:4]:
